In [1]:
import numpy as np
import pandas as pd
import random
import os
import csv
import torch
import torchvision.models as models
import torchvision.transforms as transforms

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from PIL import Image
from sklearn import preprocessing

In [2]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.RandomRotation(20),      # Rotate the image by up to 20 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),  # Randomly change brightness, contrast, and saturation
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.Resize((224, 224))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.Resize((224, 224))
])


class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = torch.from_numpy(labels).type(torch.LongTensor)
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        label = self.labels[index]
        image = Image.open(image_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label


In [4]:
label_encoder = preprocessing.LabelEncoder()

# Load pre-trained models
vgg19 = models.vgg19(pretrained=True)

toy_data_path = 'toy_dataset_label.csv'
toy_data = pd.read_csv(toy_data_path, on_bad_lines='skip', delimiter='\t',quoting=csv.QUOTE_NONE)
labels_toy_data = toy_data.iloc[:, 9].values

classes = set()
for val in labels_toy_data:
    if pd.notna(val):
        classes.add(val)
num_classes = len(classes)
vgg19.classifier[6] = torch.nn.Linear(vgg19.classifier[6].in_features, num_classes)


C:\Users\Buse\anaconda3\envs\comp411\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to C:\Users\Buse/.cache\torch\hub\checkpoints\vgg19-dcbb9e9d.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

In [19]:
#freezes up to this layer, non-inclusive
freeze_until_layer = 'features.34'

# Flag to mark when to start unfreezing layers
freeze = True

def freeze_layers(model, freeze_until_layer):
    freeze = True
    for name, module in model.named_modules():
        if name == freeze_until_layer:
            freeze = False
        for param in module.parameters():
            param.requires_grad = not freeze
    return model

vgg19 = freeze_layers(vgg19, freeze_until_layer)
            

"""# You can print out to check which layers are frozen and which are not
for name, param in vgg19.named_parameters():
    print(f"Layer: {name}, Frozen: {not param.requires_grad}")"""

'# You can print out to check which layers are frozen and which are not\nfor name, param in vgg19.named_parameters():\n    print(f"Layer: {name}, Frozen: {not param.requires_grad}")'

In [13]:
def train_model(model, train_loader, val_loader, num_epochs=5):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    criterion = torch.nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        model.train()
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f'Epoch [{epoch+1}/{num_epochs}], Training finished')
        # Validation step
        model.eval()
        total_val_loss = 0
        total_correct = 0
        total_samples = 0

        with torch.no_grad():
            for val_images, val_labels in val_loader:
                outputs = model(val_images)
                loss = criterion(outputs, val_labels)
                total_val_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                total_correct += (predicted == val_labels).sum().item()
                total_samples += val_labels.size(0)

        avg_val_loss = total_val_loss / len(val_loader)
        val_accuracy = total_correct / total_samples

        print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

    # Return the trained model
    return model

def test_model(model, test_loader):
    criterion = torch.nn.CrossEntropyLoss()
    
    model.eval()
    with torch.no_grad():
        total = 0
        correct = 0
        test_loss = 0
        criterion = torch.nn.CrossEntropyLoss()  # Replace with your loss function

        for images, labels in test_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()


        average_loss = test_loss / len(test_loader)
        accuracy = correct / total

        print(f'Test Loss: {average_loss:.4f}, Test Accuracy: {accuracy:.4f}')

In [16]:
all_image_paths = []
for i in range(200):
    img_path = 'toy_dataset/toy_dataset/{}.jpg'.format(i+1)
    all_image_paths.append(img_path)
    
all_labels = labels_toy_data[:200]
train_paths, test_paths, train_labels, test_labels = train_test_split(all_image_paths, all_labels, test_size=0.2, random_state=42)

# Further split the training set into training and validation
train_paths, val_paths, train_labels, val_labels = train_test_split(
    train_paths, train_labels, test_size=0.25, random_state=42)

train_dataset = CustomDataset(train_paths, label_encoder.fit_transform(train_labels), transform=train_transform)
val_dataset = CustomDataset(val_paths, label_encoder.fit_transform(val_labels), transform=test_transform)
test_dataset = CustomDataset(test_paths, label_encoder.fit_transform(test_labels), transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

trained_vgg19 = train_model(vgg19, train_loader, val_loader, num_epochs=5)


Epoch [1/5], Training finished
Epoch [1/5], Validation Loss: 1.7119, Validation Accuracy: 0.4750
Epoch [2/5], Training finished
Epoch [2/5], Validation Loss: 1.4145, Validation Accuracy: 0.5750
Epoch [3/5], Training finished
Epoch [3/5], Validation Loss: 1.1946, Validation Accuracy: 0.6250
Epoch [4/5], Training finished
Epoch [4/5], Validation Loss: 1.1147, Validation Accuracy: 0.6500
Epoch [5/5], Training finished
Epoch [5/5], Validation Loss: 1.0200, Validation Accuracy: 0.7500


In [17]:
test_model(trained_vgg19, test_loader)

Test Loss: 3.8608, Test Accuracy: 0.1000
